In [1]:
import pickle
import sys
sys.path.append("../")
import os
import surprise
import papermill as pm
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt

from reco_utils.common.timer import Timer
from reco_utils.evaluation.python_evaluation import rmse, mae
from reco_utils.recommender.surprise.surprise_utils import predict, compute_ranking_predictions
from surprise.model_selection import train_test_split
from surprise import Dataset
from surprise import Reader

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

System version: 3.7.6 (default, Dec 19 2019, 23:50:13) 
[GCC 7.4.0]
Surprise version: 1.1.0


In [2]:
reader = Reader(sep=',', rating_scale=(1,10))

train = Dataset.load_from_file('../Data/train.csv', reader=reader)
train, test = train_test_split(train, test_size=0.00001, random_state=42)

In [3]:
svd = surprise.SVD(random_state=42, n_factors=500, n_epochs=100, verbose=True)

with Timer() as train_time:
    svd.fit(train)

print("Took {} seconds for training.".format(train_time.interval))

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

In [4]:
test_svd = pd.read_csv('../Data/test.csv', names=['user_id', 'profile_id', 'rating'])

test_svd['user_id'] = test_svd['user_id'].astype(str)
test_svd['profile_id'] = test_svd['profile_id'].astype(str)

predictions = predict(svd, test_svd, usercol='user_id', itemcol='profile_id')

predictions.head()

,user_id,profile_id,prediction
0,0,1826,5.888675
1,0,300,4.530932
2,0,1215,3.646224
3,0,76,4.897814
4,0,842,2.389549


In [5]:
rms = sqrt(mean_squared_error(test_svd['rating'].values, predictions['prediction'].values))
rms

2.1151723153332824

In [6]:
test2 = train.build_testset()

df_test = pd.DataFrame(test2, columns=['user_id', 'profile_id', 'rating'])

predictions = predict(svd, df_test, usercol='user_id', itemcol='profile_id')
predictions.head()

,user_id,profile_id,prediction
0,307,1042,9.945288
1,307,1568,8.917613
2,307,378,7.976161
3,307,1093,8.002033
4,307,1127,9.765299


In [7]:
rms = sqrt(mean_squared_error(df_test['rating'].values, predictions['prediction'].values))
rms

0.1235336368791968

In [8]:
f = open('model/SVD.pickle', 'wb')
pickle.dump(svd, f)  
f.close()

In [9]:
f = open('model/SVD.pickle','rb') 
loaded_model = pickle.load(f)
f.close()

In [10]:
predictions = predict(loaded_model, df_test, usercol='user_id', itemcol='profile_id')
predictions.head()

,user_id,profile_id,prediction
0,307,1042,9.945288
1,307,1568,8.917613
2,307,378,7.976161
3,307,1093,8.002033
4,307,1127,9.765299


In [11]:
rms = sqrt(mean_squared_error(df_test['rating'].values, predictions['prediction'].values))
rms

0.1235336368791968